In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import numpy as np
import re
import string

In [3]:
institutional_type = pd.read_csv('../../data/institutions_us.csv')

In [4]:
address = pd.read_pickle('/data/datasets/WOS/US/Adresses_impact.p')

In [5]:
carnegie_df = pd.read_excel('../../data/carnegie/CCIHE2021-PublicData.xlsx',sheet_name='Data',
                           converters = {'unitid':str,'selindex':str})
# carnegie_df = carnegie_df[['unitid','name']]
# carnegie_df.columns = ['carnegie_id','carnegie_name']

# carnegie_df['carnegie_id'] = carnegie_df.carnegie_id.astype("category")

In [6]:
handcoding_df = pd.read_excel('../../data/institutions_20220329.xlsx',sheet_name='Sheet1',converters={'index':int,'carnegie_id':int})

handcoding_df['carnegie_id'] = handcoding_df.carnegie_id.astype('str')

handcoding_df.drop(columns='Unnamed: 0', inplace=True)

In [7]:
handcoding_flags = pd.read_excel('../../handcoding/carnegie_flags.xlsx',sheet_name='to_clean',converters={'real_unitid':str})

handcoding_flags = handcoding_flags.dropna(subset=['real_unitid']).drop_duplicates()

In [8]:
real_hsi = carnegie_df[carnegie_df.hsi==1].unitid.unique()

In [9]:
len(real_hsi)

332

In [10]:
handcoding_flags.potential_group.value_counts()

hsi     250
hbcu     59
wc       28
Name: potential_group, dtype: int64

In [11]:
#we remove:
handcoding_flags[(handcoding_flags.potential_group == 'hsi') & (-handcoding_flags.real_unitid.isin(real_hsi))]
#we keep
handcoding_flags = handcoding_flags[
    (handcoding_flags.potential_group == 'hbcu') |
    (handcoding_flags.potential_group == 'wc') |
    ((handcoding_flags.potential_group == 'hsi') & (handcoding_flags.real_unitid.isin(real_hsi)))]

In [12]:
handcoding_flags.potential_group.value_counts()

hsi     100
hbcu     59
wc       28
Name: potential_group, dtype: int64

In [13]:
handcoding_flags = handcoding_flags[['name_wos','n','real_unitid']]
handcoding_flags.columns = ['name','n','carnegie_id']

In [14]:
handcoding_flags = handcoding_flags.merge(carnegie_df[['unitid','name']].rename(columns={"unitid":'carnegie_id','name':'carnegie_name'}), on='carnegie_id')


In [15]:
handcoding_flags['sector']= 'university'
handcoding_flags['grid_id'] = ''
handcoding_flags['grid_name'] = ''
handcoding_flags['Handcoder notes'] = 'special handcoding hbcu-hsi-wc'
handcoding_flags['new_name'] = handcoding_flags.name
handcoding_flags['Check'] = 'ok'


In [16]:
handcoding_df = handcoding_df.append(handcoding_flags)

In [17]:
handcoding_df = handcoding_df[handcoding_df.Check == 'ok']

In [18]:
handcoding_df[['name','new_name','grid_id','grid_name','carnegie_id','carnegie_name']].to_csv('../../data/institutions_crosswalk.csv',index=False)

In [19]:
address_clean = address.merge(handcoding_df[['name','grid_id','carnegie_id','carnegie_name']],how='inner',left_on = 'Institution',right_on='name')

In [20]:
print('original number of rows: ' + str(address.shape[0]/1000000) + ' millions')
print('number of rows after cleaning: ' + str(address_clean.shape[0]/1000000) + ' millions')

original number of rows: 21.290383 millions
number of rows after cleaning: 16.03094 millions


In [21]:
print('original number of rows: ' + str(address.shape[0]/1000000) + ' millions')
print('number of rows after cleaning: ' + str(address_clean.shape[0]/1000000) + ' millions')

original number of rows: 21.290383 millions
number of rows after cleaning: 16.03094 millions


In [22]:
address.sector.value_counts(dropna=False)

university    14177659
hospital       3532425
goverment      1747917
company        1152552
NaN             679830
Name: sector, dtype: int64

In [23]:
21290383*0.665918

14177649.266594

### Carnegie classifications

In [24]:
carnegie_df = carnegie_df[['unitid','name', 'basic2021','control','obereg','hbcu','tribal','hsi','msi','womens','selindex']]

In [25]:
carnegie_df['r1']='not R1'

carnegie_df.loc[carnegie_df.basic2021 == 15,'r1'] = 'R1'

In [26]:
carnegie_df['control'] = carnegie_df.control.replace({1:'public',2:'private_nfp',3:'private_fp'})

carnegie_df['obereg'] = carnegie_df.obereg.replace({0: "US_Service_schools",
                                                     1: "New_England",
                                                     2: "Mid_East",
                                                     3: "Great_Lakes",
                                                     4: "Plains",
                                                     5: "Southeast",
                                                     6: "Southwest",
                                                     7: "Rocky_Mountains",
                                                     8: "Far_West",
                                                     9: "Outlying_areas"})

carnegie_df['hbcu'] = carnegie_df.hbcu.replace({1:'HBCU', 2:'not HBCU'})
carnegie_df['tribal'] = carnegie_df.tribal.replace({1:'tribal', 2:'not tribal'})
carnegie_df['hsi'] = carnegie_df.hsi.replace({0:'not HSI', 1:'HSI'})
carnegie_df['msi'] = carnegie_df.msi.replace({0:'not MSI', 1:'MSI'})
carnegie_df['womens'] = carnegie_df.womens.replace({0:'not womens', 1:'Womens'})

In [27]:
carnegie_df.selindex.replace({'1':'inclusive','2':'selective','3':'more_selective'}).value_counts(dropna=False)

NaN               1288
inclusive         1268
selective          562
0                  525
more_selective     296
Name: selindex, dtype: int64

### Most institutions don't have a selectivity index

In [28]:
carnegie_df['selindex'] = carnegie_df.selindex.replace({'0':'not_indexed',np.NaN:'not_indexed',
                              '1':'inclusive','2':'selective','3':'more_selective'})

In [29]:
carnegie_df = carnegie_df[['unitid','r1','control', 'obereg', 'hbcu', 'tribal',
       'hsi', 'msi', 'womens', 'selindex']]

In [30]:
carnegie_df.r1.value_counts()

not R1    3793
R1         146
Name: r1, dtype: int64

In [31]:
address_clean = address_clean.merge(carnegie_df, left_on='carnegie_id',right_on='unitid')

In [32]:
address_clean.r1.value_counts()

R1        13416187
not R1     2614753
Name: r1, dtype: int64

In [33]:
address_clean = address_clean[['cluster_id', 'id_art', 'grid_id', 'carnegie_id',
       'carnegie_name', 'r1', 'control', 'obereg', 'hbcu', 'tribal',
       'hsi', 'msi', 'womens', 'selindex']]

In [34]:
address_clean.shape

(16030940, 14)

In [35]:
address_clean.drop_duplicates().shape

(16026519, 14)

In [36]:
address_clean = address_clean.drop_duplicates()

### Remove institutions that don't follow the selection criteria:

1. being HSI, HBCU, WC 
OR
2. having more than 500 papers on the database

In [37]:
address_clean.columns

Index(['cluster_id', 'id_art', 'grid_id', 'carnegie_id', 'carnegie_name', 'r1',
       'control', 'obereg', 'hbcu', 'tribal', 'hsi', 'msi', 'womens',
       'selindex'],
      dtype='object')

In [38]:
n_papers_on_WOS = handcoding_df.groupby('carnegie_id').n.sum().to_frame().rename(columns={'n':'n_papers_on_WOS'})

In [39]:
address_clean = address_clean.merge(n_papers_on_WOS, on='carnegie_id')

In [40]:
address_clean = address_clean[
    (address_clean.hsi == 'HSI') |
    (address_clean.hbcu == 'HBCU') |
    (address_clean.womens == 'Womens') |
    (address_clean.n_papers_on_WOS>=500)]

In [41]:
address_clean.to_pickle('/data/datasets/WOS/US/address_clean_carnegie.p')